In [1]:
# make the notebook automatically reload external python modules
%load_ext autoreload
%autoreload 2

In [2]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.


In [17]:
from llm import LLM
from input_file import Input

import json

In [18]:
AZURE_OPENAI_API_KEY='e865ffe4387f416ea57d7394ee09fb2e'
AZURE_OPENAI_ENDPOINT='https://openaifor3267.openai.azure.com/'
AZURE_OPENAI_API_VERSION = '2023-12-01-preview'
MODEL = "gpt-35-16k"

In [19]:
llm = LLM(AZURE_OPENAI_API_KEY, AZURE_OPENAI_ENDPOINT, AZURE_OPENAI_API_VERSION, MODEL)

In [20]:
# Program flow - steps 1 and 3
input_file = Input('input.txt')

In [21]:
class SampleServices:
    """ Implementation of program flow parts relating to a single sample """

    UNIT_TESTS_OUTPUT_VAR = 'unit_tests_output'

    def __init__(self, input_test: Input):
        self.input = input_test
        self.unit_tests_code = f'\n\n{SampleServices.UNIT_TESTS_OUTPUT_VAR} = [\n\t' + \
                               ',\n\t'.join(input_test.unit_tests) + \
                               '\n]'
        self.n_unit_tests = len(input_test.unit_tests)

    def add_unit_tests(self, sample: str) -> str:
        """ Step 6 """
        return sample + self.unit_tests_code

    def save_sample(self, sample: str, filename_format: str, sample_number, temperature) -> str:
        """ Step 7 """
        filename = filename_format.format(name=self.input.name, sample_number = sample_number + 1, temp = int(temperature*10))
        with open(filename, "w") as f_sample:
            f_sample.write(sample)
        return filename

    def test_sample(self, sample: str, filename: str) -> int:
        """ Step 8
        Returns number of passed unit tests
        """
        try:
            local_vars = {}  # isolate from current program
            exec(sample, local_vars, local_vars)  # may print if generated code includes prints (or have other side effects)
            unit_tests_output = local_vars[SampleServices.UNIT_TESTS_OUTPUT_VAR]
        except Exception as ex:
            print(f'Running generated code {filename} failed: {ex}')
            return 0

        assert len(unit_tests_output) == len(self.input.ground_truth), 'Wrong number of unit tests run'
        # Count number of tests which returned correct result
        n_passed_tests = sum(1 for actual, expected in zip(unit_tests_output, self.input.ground_truth) if str(actual) == expected)
        return n_passed_tests

    def append_stats(self, f, insert: str, n_passed_tests: int):
        """ Step 9 """
        f.write(f'passed_{insert}_examples={n_passed_tests}\n'
                f'failed_{insert}_examples={self.n_unit_tests - n_passed_tests}\n'
                f'average_{insert}_examples={(n_passed_tests / self.n_unit_tests):.2f}\n\n')

In [22]:
sample_svc = SampleServices(input_file)

In [23]:
# For Step 5 - just instruct LLM to ensure no trailing or leading text
SYSTEM_CONTENT_NO_EXAMPLES  = 'Generate complete and correct code implementing the Python function below. Code only, not extra strings or explanations please. Comment extensively inside the code.'
user_content_no_examples = f'Function name: \"{input_file.name}\".\n' \
                           f'Function signature: {input_file.signature}\n' \
                           f'Function description: {input_file.description}'

FILENAME_FORMAT_NO_EXAMPLES = '{name}-s{sample_number}-t{temp}.py'

print(f'BASIC PROMPT:\n{SYSTEM_CONTENT_NO_EXAMPLES}\n{user_content_no_examples}')

BASIC PROMPT:
Generate complete and correct code implementing the Python function below. Code only, not extra strings or explanations please. Comment extensively inside the code.
Function name: "sqrt_list".
Function signature: sqrt_list(l:list) -> float
Function description: A function named sqrt_list that takes a list of integer and returns the square root of the sum of that list, to two decimal places.


In [24]:
# Step 4
SYSTEM_CONTENT_WITH_EXAMPLES  = f'{SYSTEM_CONTENT_NO_EXAMPLES} Be sure that provided examples pass.'

unit_tests_to_text = ',\n'.join(f'{unit_test} must return {ground_truth}' for unit_test, ground_truth in zip(input_file.unit_tests, input_file.ground_truth))
user_content_with_examples = f'{user_content_no_examples}\nExamples:\n{unit_tests_to_text}.'

FILENAME_FORMAT_WITH_EXAMPLES = '{name}-ex-s{sample_number}-t{temp}.py'

print(f'PROMPT WITH EXAMPLES:\n{SYSTEM_CONTENT_WITH_EXAMPLES}\n{user_content_with_examples}')

PROMPT WITH EXAMPLES:
Generate complete and correct code implementing the Python function below. Code only, not extra strings or explanations please. Comment extensively inside the code. Be sure that provided examples pass.
Function name: "sqrt_list".
Function signature: sqrt_list(l:list) -> float
Function description: A function named sqrt_list that takes a list of integer and returns the square root of the sum of that list, to two decimal places.
Examples:
sqrt_list([6,18,7,5]) must return 6.0,
sqrt_list([50,2,33,8,4,3,10]) must return 10.48.


In [25]:
def prompt_flow(system_content: str, user_content: str, temperature: float, filename_format: str, insert: str, file_stats):
    """ Program flow for 1 temperature and 1 prompt"""
    generated_samples = llm.invoke(system_content, user_content, temperature, input_file.k)
    for sample_number, sample in enumerate(generated_samples):
        # 9
        file_stats.write(f'LLM={llm.model}\n'
                      f'temperature={temperature}\n')
        # 6
        p = sample_svc.add_unit_tests(sample)
        # 7
        sample_filename = sample_svc.save_sample(p, filename_format, sample_number=sample_number + 1, temperature=temperature)
        # 8
        n_passed_tests = sample_svc.test_sample(p, sample_filename)
        # 9
        sample_svc.append_stats(file_stats, insert, n_passed_tests)

In [26]:
# RUN THE PROGRAM FLOW
with open(f'{input_file.name}-stats.txt', 'w') as f_stats:
    for t in input_file.temperatures:
        prompt_flow(SYSTEM_CONTENT_NO_EXAMPLES, user_content_no_examples, t, FILENAME_FORMAT_NO_EXAMPLES, "without", f_stats)
        prompt_flow(SYSTEM_CONTENT_WITH_EXAMPLES, user_content_with_examples, t, FILENAME_FORMAT_WITH_EXAMPLES, "with", f_stats)